In [2]:
import csv
import os
import pandas as pd

In [32]:
df = pd.read_csv('movie.csv')[0:1]
print(df)
print('--------------')
df['key']=df['中文名稱'].astype(str)+df['類型'].astype(str)+df['簡介']
df


   Unnamed: 0        中文名稱                       英文名稱        上映日期      類型  \
0           0  小小兵2：格魯的崛起  Minions: The Rise of Guru  2022-07-01  動畫動作喜劇   

                                                  簡介  
0  今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，...  
--------------


,Unnamed: 0,中文名稱,英文名稱,上映日期,類型,簡介,key
0,0,小小兵2：格魯的崛起,Minions: The Rise of Guru,2022-07-01,動畫動作喜劇,今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將帶來關於一個12歲小男孩的故事，...,小小兵2：格魯的崛起動畫動作喜劇今年暑假，影史最賣座的動畫電影系列之一和風靡全球的現象，即將...


In [26]:
import pandas as pd
df1 = pd.DataFrame([['Snow','2',22],['Tyrion','2',32],['Sansa','0',18],['Arya','0',14]], columns=['name','gender','age'])

print("--------更换单个值----------")
# loc和iloc 可以更换单行、单列、多行、多列的值
df1.loc[0,'age']=25      # 思路：先用loc找到要更改的值，再用赋值（=）的方法实现更换值
df1.iloc[0,2]=25         # iloc：用索引位置来查找

# at 、iat只能更换单个值
df1.at[0,'age']=25      # iat 用来取某个单值,参数只能用数字索引
df1.iat[0,2]=25         # at 用来取某个单值,参数只能用index和columns索引名称
print(df1)

df1['score']=df1['age'].astype(str)+df1['gender'].astype(str)
print(df1)

--------更换单个值----------
     name gender  age
0    Snow      2   25
1  Tyrion      2   32
2   Sansa      0   18
3    Arya      0   14
     name gender  age score
0    Snow      2   25   252
1  Tyrion      2   32   322
2   Sansa      0   18   180
3    Arya      0   14   140
